In [2]:
import pandas as pd
from utils import RAW_DATA_DIR
import ast

In [3]:
def load(filepath):
    tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

    COLUMNS = [("track", "tags"), ("album", "tags"), ("artist", "tags"), ("track", "genres"), ("track", "genres_all")]
    for column in COLUMNS:
        tracks[column] = tracks[column].map(ast.literal_eval)

    COLUMNS = [
        ("track", "date_created"),
        ("track", "date_recorded"),
        ("album", "date_created"),
        ("album", "date_released"),
        ("artist", "date_created"),
        ("artist", "active_year_begin"),
        ("artist", "active_year_end"),
    ]
    for column in COLUMNS:
        tracks[column] = pd.to_datetime(tracks[column])

    SUBSETS = ("small", "medium", "large")
    try:
        tracks["set", "subset"] = tracks["set", "subset"].astype("category", categories=SUBSETS, ordered=True)
    except (ValueError, TypeError):
        # the categories and ordered arguments were removed in pandas 0.25
        tracks["set", "subset"] = tracks["set", "subset"].astype(pd.CategoricalDtype(categories=SUBSETS, ordered=True))

    COLUMNS = [
        ("track", "genre_top"),
        ("track", "license"),
        ("album", "type"),
        ("album", "information"),
        ("artist", "bio"),
    ]
    for column in COLUMNS:
        tracks[column] = tracks[column].astype("category")

    return tracks

In [34]:
df = load(RAW_DATA_DIR / "fma_metadata" / "tracks.csv")

In [15]:
df.columns

MultiIndex([( 'album',          'comments'),
            ( 'album',      'date_created'),
            ( 'album',     'date_released'),
            ( 'album',          'engineer'),
            ( 'album',         'favorites'),
            ( 'album',                'id'),
            ( 'album',       'information'),
            ( 'album',           'listens'),
            ( 'album',          'producer'),
            ( 'album',              'tags'),
            ( 'album',             'title'),
            ( 'album',            'tracks'),
            ( 'album',              'type'),
            ('artist', 'active_year_begin'),
            ('artist',   'active_year_end'),
            ('artist', 'associated_labels'),
            ('artist',               'bio'),
            ('artist',          'comments'),
            ('artist',      'date_created'),
            ('artist',         'favorites'),
            ('artist',                'id'),
            ('artist',          'latitude'),
          

In [46]:
df[df["album"]["tags"].map(lambda x: len(x)) != 0]

album                                                        \
         comments        date_created date_released  engineer favorites   
track_id                                                                  
992             0 2008-11-26 02:29:10           NaT       NaN         0   
4119            0 2008-12-04 09:29:00    1995-12-24       NaN         1   
5477            0 2009-01-30 07:24:01    2004-01-25       NaN         8   
5510            0 2009-01-30 07:24:01    2004-01-25       NaN         8   
5570            0 2009-01-30 07:24:01    2004-01-25       NaN         8   
...           ...                 ...           ...       ...       ...   
154897          0 2017-03-19 06:16:11    2017-03-19       NaN         0   
155000          1 2017-03-21 22:56:31    2017-03-22       NaN         0   
155063          0 2017-03-24 19:40:34    2017-03-24  JBlanked         1   
155064          0 2017-03-24 19:40:34    2017-03-24  JBlanked         1   
155065          0 2017-03-24 19:40:34    2017-03-24  JBlanked         1   

                                                                            \
             id                                        information listens   
track_id                                                                     
992         252                                                NaN    4404   
4119       1460  <p>"Christmas With Quintron - Mr. Quintron, Mi...    4507   
5477       1889  <p>All content unwritten, unplayed, unsung and...    7851   
5510       1889  <p>All content unwritten, unplayed, unsung and...    7851   
5570       1889  <p>All content unwritten, unplayed, unsung and...    7851   
...         ...                                                ...     ...   
154897    22861  <p>Deep 3D turntable spinning like a phat bitc...    2529   
155000    22880  <p>Some upbeat ukulele lounge loafering.</p>\n...    1208   
155063    22899  <p>Hip Hop, Old School type beat 2017, Trap Mu...    4524   
155064    22899  <p>Hip Hop, Old School type beat 2017, Trap Mu...    4524   
155065    22899  <p>Hip Hop, Old School type beat 2017, Trap Mu...    4524   

                              \
                    producer   
track_id                       
992                      NaN   
4119                   Stork   
5477                     NaN   
5510                     NaN   
5570                     NaN   
...                      ...   
154897                   NaN   
155000    Andre William owen   
155063              JBlanked   
155064              JBlanked   
155065              JBlanked   

                                                             ...       track  \
                                                       tags  ... information   
track_id                                                     ...               
992                                             [baltimore]  ...         NaN   
4119                                          [new orleans]  ...         NaN   
5477      [children, jim reeves, youth performance, coun...  ...         NaN   
5510      [children, jim reeves, youth performance, coun...  ...         NaN   
5570      [children, jim reeves, youth performance, coun...  ...         NaN   
...                                                     ...  ...         ...   
154897    [electronic, italian, edm, spettro records, el...  ...         NaN   
155000    [beatnik, hip, turtleneck, swagger, foxtrot, b...  ...         NaN   
155063    [old school beats, 2017 free instrumentals, fr...  ...         NaN   
155064    [old school beats, 2017 free instrumentals, fr...  ...         NaN   
155065    [old school beats, 2017 free instrumentals, fr...  ...         NaN   

                                 \
         interest language_code   
track_id                          
992          1654            en   
4119          798            en   
5477         3476            en   
5510         1822            en   
5570         1248            en   
...           ... 

In [37]:
df = df[df["set"]["subset"] == "medium"]
(df["album"]["information"] == "<p></p>").sum()

np.int64(27)

In [29]:
df = df[df["set"]["subset"] == "medium"]
df = df[~df["album"]["information"].isna()]["album"].head(10)
df

In [54]:
from bs4 import BeautifulSoup
import re

In [56]:
def sanitize_text(html_string):
    # Remove all <a> tags and their content
    soup = BeautifulSoup(html_string, "html.parser")
    clean_text = soup.get_text(separator=" ", strip=True)
    url_pattern = re.compile(r"(https?://\S+|www\.\S+)", re.IGNORECASE)
    clean_text = url_pattern.sub("", clean_text)
    # Remove extra whitespace
    clean_text = re.sub(r"\s+", " ", clean_text).strip()
    return clean_text

In [58]:
df["album"]["information"].apply(sanitize_text).isna().sum()

/tmp/ipykernel_434701/4021734980.py:3: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(html_string, "html.parser")


np.int64(3218)

In [55]:
html_string = df["album"].iloc[-1]["information"]
url_pattern = re.compile(r"(https?://\S+|www\.\S+)", re.IGNORECASE)
html_string = url_pattern.sub("", html_string)
soup = BeautifulSoup(html_string, "html.parser")
soup.get_text(strip=True)

'A live performance at Monty Hall on Feb 17, 2017 on a bill with Screaming Females. For the full video of their performance, please visit'

In [59]:
df["track"]

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
134,256000,0,NaN,2008-11-26 01:43:19,2008-11-26,207,3,Hip-Hop,[21],[21],NaN,1126,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,943,NaN,5,NaN,[],Street Music
136,256000,1,NaN,2008-11-26 01:43:35,2008-11-26,509,0,Rock,"[45, 58]","[58, 12, 45]",NaN,1948,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1498,NaN,0,NaN,[],Peel Back The Mountain Sky
139,128000,0,NaN,2008-11-26 01:44:05,2008-11-26,296,3,Folk,[17],[17],NaN,702,en,Attribution-Noncommercial-No Derivative Works ...,582,NaN,2,NaN,[],CandyAss
181,256000,0,NaN,2008-11-26 01:46:31,2006-01-01,171,8,Rock,[27],"[27, 12]",NaN,1736,en,Attribution-Noncommercial-No Derivative Works ...,1339,NaN,1,NaN,[],Gopacapulco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155297,320000,0,NaN,2017-03-30 09:52:59,NaT,252,3,Instrumental,"[18, 107, 1235]","[107, 18, 1235]",NaN,1463,NaN,Attribution-NonCommercial,1049,NaN,9,NaN,[],Nebula Reborn
155298,320000,0,NaN,2017-03-30 10:47:51,NaT,151,0,Folk,"[17, 103]","[17, 103]",NaN,706,NaN,Attribution,590,NaN,2,NaN,[],An Idiot Abroad
155306,320000,0,NaN,2017-03-30 10:48:02,NaT,162,1,Folk,"[17, 103]","[17, 103]",NaN,497,NaN,Attribution,435,NaN,1,NaN,[],Tiny Man


In [12]:
df.columns

MultiIndex([( 'album',          'comments'),
            ( 'album',      'date_created'),
            ( 'album',     'date_released'),
            ( 'album',          'engineer'),
            ( 'album',         'favorites'),
            ( 'album',                'id'),
            ( 'album',       'information'),
            ( 'album',           'listens'),
            ( 'album',          'producer'),
            ( 'album',              'tags'),
            ( 'album',             'title'),
            ( 'album',            'tracks'),
            ( 'album',              'type'),
            ('artist', 'active_year_begin'),
            ('artist',   'active_year_end'),
            ('artist', 'associated_labels'),
            ('artist',               'bio'),
            ('artist',          'comments'),
            ('artist',      'date_created'),
            ('artist',         'favorites'),
            ('artist',                'id'),
            ('artist',          'latitude'),
          

In [ ]:
import os


def get_audio_path(audio_dir, track_id):
    """
    Return the path to the mp3 given the directory where the audio is stored
    and the track ID.

    Examples
    --------
    >>> import utils
    >>> AUDIO_DIR = os.environ.get('AUDIO_DIR')
    >>> utils.get_audio_path(AUDIO_DIR, 2)
    '../data/fma_small/000/000002.mp3'

    """
    tid_str = "{:06d}".format(track_id)
    return os.path.join(audio_dir, tid_str[:3], tid_str + ".mp3")